###### Install/Import Dependencies

In [1]:
# # install API dependency (if needed)
# pip install kaggle
# print(f'Kaggle has been installed!')
# print(f'-')

In [2]:
# import database connection configuration
# must create config.py in project root (currently on .gitignore)
from config import user, password, host, database, param_dic

# import python dependencies
import kaggle
import numpy as np
import os
import pandas as pd
import psycopg2
import psycopg2.extras as extras
from sqlalchemy import create_engine
import sys
import time

# print import status
print(f'Database configuration imported')
print(f'Python dependencies imported')

Database configuration imported
Python dependencies imported


In [3]:
# record start time for total run time
nb_start = time.time() # measures number of seconds since 1970

# print time record status
print(f'Process start time recorded')
print(f'---')

Process start time recorded
---


In [4]:
# Go to https://www.kaggle.com/user_name/account to create API token
# Place kaggle.json (download) into 'C:\Users\computer_name\.kaggle\kaggle.json'
kaggle.api.authenticate()

# print token status
print(f'Kaggle API token loaded')

Kaggle API token loaded


In [5]:
print(f'Retrieving Brain Weight data from Kaggle...')

# download kaggle dataset (NHL Hockey Data)
# will overwrite local files, if they exist
kaggle.api.dataset_download_files(
    'anubhabswain/brain-weight-in-humans',
    path = '../big-brained-brads/data/',
    unzip = True
)

# print file retrieval status
print(f'Files retrieved!')
print(f'-')

Retrieving Brain Weight data from Kaggle...
Files retrieved!
-


###### Import CSVs to Dataframes, Transform

In [6]:
# print import process message
print(f'Begin importing CSV to dataframe...')

Begin importing CSV to dataframe...


In [7]:
# read player_info csv to df
brains_df = pd.read_csv('data/dataset.csv')
brains_df.rename(columns = {
    'Gender' : 'gender',
    'Age Range' : 'age',
    'Head Size(cm^3)' : 'size',
    'Brain Weight(grams)' : 'weight'
}, inplace = True)
brains_df.head(5)

,gender,age,size,weight
0,1,1,4512,1530
1,1,1,3738,1297
2,1,1,4261,1335
3,1,1,3777,1282
4,1,1,4177,1590


In [8]:
brains_df['gender'] = brains_df['gender'].astype(str)
brains_df['age'] = brains_df['age'].astype(str)

In [9]:
brains_df.dtypes

gender    object
age       object
size       int64
weight     int64
dtype: object

In [10]:
brains_df['gender'].replace({'1' : 'male', '2' : 'female'}, inplace = True)
brains_df['age'].replace({'1' : 'adult', '2' : 'child'}, inplace = True)
brains_df.head()

,gender,age,size,weight
0,male,adult,4512,1530
1,male,adult,3738,1297
2,male,adult,4261,1335
3,male,adult,3777,1282
4,male,adult,4177,1590


###### Connect to Database

In [11]:
# create connection function using psycops execute(), also print statuses
def connect(params):
    conn = None
    try:
        print(f'Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
        # create cursor to run query
        cur = conn.cursor()
        # return database version
        cur.execute('SELECT version();')
        # fetch one result
        record = cur.fetchone()
        print('You are connected to -', record)
        # close cursor
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print('Error while connecting to PostgreSQL database', error)
    # close connection
    finally:
        if (conn):
            cur.close()
            conn.rollback()
        return conn

In [12]:
# create connector for pandas.to_sql() method for loading data to database
engine = create_engine(f'postgresql://{user}:{password}@{host}:5432/{database}')

# print connector status
print(f'Database connector created')

Database connector created


In [13]:
# test database connection
conn = connect(param_dic)

# print partition from connection status
print(f'-')

Connecting to the PostgreSQL database...
You are connected to - ('PostgreSQL 13.2, compiled by Visual C++ build 1914, 64-bit',)
-


In [14]:
# create schema function with status print
def create_schema(sql_file):
    cursor = conn.cursor()
    try:
        print(f'Creating schema in database using {sql_file}...')
        cursor.execute(open(sql_file, 'r').read())
        print('Schema successfully created!')
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print('Error while creating schema in database', error)
        conn.rollback()
    # close cursor
    cursor.close()

In [15]:
# run SQL file to create schema
create_schema('queries/create_schema.sql')

Creating schema in database using queries/create_schema.sql...
Schema successfully created!


In [16]:
# to sql function to use pandas to_sql function with status print
def to_sql(df, table):
    start = time.time()
    print(f'Staging {table}...')
    print(f'-')
    try:
        print(f'Loading {table} to the PostgreSQL database...')
        # use pd.to_sql() to set load details
        df.to_sql(
            table, 
            con = engine, 
            schema = 'public', 
            if_exists = 'append',
            index = False
        )
    except (Exception, psycopg2.DatabaseError) as error:
        print(f'Error: %s' % error)
        print(f'-')
        print(f'Load failed.')
        print(f'---')
        return 1
    end = time.time()
    seconds = round((end - start) % 60)
    minutes = round(((end - start) - seconds) / 60)
    print(f'Load successful!  Run Time: {minutes}min {seconds}sec')
    print(f'---')

In [17]:
# print data load status message
print(f'-')
print(f'Begin loading dataframes to the PostgreSQL database.')
print(f'-')

-
Begin loading dataframes to the PostgreSQL database.
-


In [18]:
# load df to sql db table (df, table)
to_sql(brains_df, 'brain_weights')

Staging brain_weights...
-
Loading brain_weights to the PostgreSQL database...
Load successful!  Run Time: 0min 0sec
---


In [19]:
# load data function for reading dataframe, loading to sql database with status print
def load_data(conn, df, table):
    start = time.time()
    print(f'Staging {table}...')
    print(f'-')
    # create list of tupples from dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute insert
    query  = 'INSERT INTO %s(%s) VALUES %%s' % (table, cols)
    cursor = conn.cursor()
    try:
        print(f'Loading {table} to the PostgreSQL database...')
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(f'Error: %s' % error)
        print(f'-')
        print(f'Load failed.')
        print(f'---')
        conn.rollback()
        cursor.close()
        return 1
    end = time.time()
    seconds = round((end - start) % 60)
    minutes = round(((end - start) - seconds) / 60)
    print(f'Load successful!  Run Time: {minutes}min {seconds}sec')
    print(f'---')
    cursor.close()

In [20]:
# load df to sql db table
load_data(conn, brains_df, 'brain_weights')

Staging brain_weights...
-
Loading brain_weights to the PostgreSQL database...
Load successful!  Run Time: 0min 0sec
---
